In [1]:
# hide
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [2]:
# default_exp preprocessing

<IPython.core.display.Javascript object>

# Preprocessing
> Feature/target selection, engineering and manipulation.

## Overview
This section provides functionality for all data manipulation steps that are needed before data is passed into a model for prediction. We group all these steps under Preprocessing. This includes feature/target selection, feature/target engineering and feature/target manipulation.

Some preprocessors work with both Pandas DataFrames and NumerFrames. Most preprocessors use specific `NumerFrame` functionality.

In the last section we explain how you can implement your own Preprocessor that integrates well with the rest of this framework.

In [3]:
# hide
from nbdev.showdoc import *

<IPython.core.display.Javascript object>

In [4]:
# export
import os
import time
import warnings
import numpy as np
import pandas as pd
import datetime as dt
from tqdm.auto import tqdm
from functools import wraps
from typeguard import typechecked
from abc import ABC, abstractmethod
from rich import print as rich_print
from typing import Union, List, Tuple
from multiprocessing.pool import Pool
from sklearn.preprocessing import QuantileTransformer 

from numerblox.numerframe import NumerFrame, create_numerframe

<IPython.core.display.Javascript object>

## 0. Base

These objects will provide a base for all pre- and post-processing functionality and log relevant information.

## 0.1. BaseProcessor

`BaseProcessor` defines common functionality for `preprocessing` and `postprocessing` (Section 5).

Every Preprocessor should inherit from `BaseProcessor` and implement the `.transform` method.

In [5]:
# export
class BaseProcessor(ABC):
    """Common functionality for preprocessors and postprocessors."""

    def __init__(self):
        ...

    @abstractmethod
    def transform(
        self, dataf: Union[pd.DataFrame, NumerFrame], *args, **kwargs
    ) -> NumerFrame:
        ...

    def __call__(
        self, dataf: Union[pd.DataFrame, NumerFrame], *args, **kwargs
    ) -> NumerFrame:
        return self.transform(dataf=dataf, *args, **kwargs)

<IPython.core.display.Javascript object>

## 0.2. Logging

We would like to keep an overview of which steps are done in a data pipeline and where processing bottlenecks occur.
The decorator below will display for a given function/method:
1. When it has finished.
2. What the output shape of the data is.
3. How long it took to finish.

To use this functionality, simply add `@display_processor_info` as a decorator to the function/method you want to track.

We will use this decorator throughout the pipeline (`preprocessing`, `model` and `postprocessing`).

Inspiration for this decorator: [Calmcode Pandas Pipe Logs](https://calmcode.io/pandas-pipe/logs.html)

In [6]:
# export
def display_processor_info(func):
    """Fancy console output for data processing."""

    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        class_name = func.__qualname__.split(".")[0]
        rich_print(
            f":white_check_mark: Finished step [bold]{class_name}[/bold]. Output shape={result.shape}. Time taken for step: [blue]{time_taken}[/blue]. :white_check_mark:"
        )
        return result

    return wrapper

<IPython.core.display.Javascript object>

In [7]:
# hide_input
class TestDisplay:
    """
    Small test for logging.
    Output should mention 'TestDisplay',
    Return output shape of (10, 314) and
    time taken for step should be close to 2 seconds.
    """

    def __init__(self, dataf: NumerFrame):
        self.dataf = dataf

    @display_processor_info
    def test(self) -> NumerFrame:
        time.sleep(2)
        return self.dataf


dataf = create_numerframe("test_assets/mini_numerai_version_1_data.csv")
TestDisplay(dataf).test()

✅ Finished step TestDisplay. Output shape=(10, 314). Time taken for step: 0:00:02.000124. ✅

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.50,0.25,0.25,...,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75,0.50
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.50,0.00,0.00,...,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00,0.25
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.00,0.75,0.75,...,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75,0.25
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.50,0.25,0.25,...,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00,0.25
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.00,0.25,0.50,...,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75,0.75
5,n009aa2d32389eca,era1,train,0.50,0.50,0.25,0.25,0.75,0.75,0.75,...,0.75,0.75,0.00,0.00,0.75,0.50,0.00,0.25,0.00,0.50
6,n009ef1a5fe009b6,era1,train,0.50,0.25,0.25,0.75,1.00,1.00,1.00,...,1.00,1.00,0.50,0.50,0.75,0.50,0.50,0.50,1.00,0.25
7,n00ae5d51f55fb0f,era1,train,0.25,1.00,1.00,0.75,1.00,0.75,0.75,...,0.50,0.25,0.75,0.75,0.00,0.25,0.75,0.50,0.25,0.25
8,n00b0ac86d77aed7,era1,train,0.50,0.50,0.50,1.00,1.00,0.25,0.50,...,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.50
9,n00c63366aeaf76a,era1,train,0.50,1.00,1.00,0.25,0.75,0.25,0.25,...,0.00,0.00,1.00,1.00,0.75,0.50,1.00,1.00,0.75,0.75


<IPython.core.display.Javascript object>

## 1. Common preprocessing steps


This section implements commonly used preprocessing for Numerai. We invite the Numerai community to develop new preprocessors.

## 1.0 Tournament agnostic

Preprocessors that can be applied for both Numerai Classic and Numerai Signals.

### 1.0.1. CopyPreProcessor

The first and obvious preprocessor is copying, which is implemented as a default in `ModelPipeline` (Section 4) to avoid manipulation of the original DataFrame or `NumerFrame` that you load in.

In [8]:
# export
@typechecked
class CopyPreProcessor(BaseProcessor):
    """Copy DataFrame to avoid manipulation of original DataFrame."""

    def __init__(self):
        super().__init__()

    @display_processor_info
    def transform(self, dataf: Union[pd.DataFrame, NumerFrame]) -> NumerFrame:
        return NumerFrame(dataf.copy())

<IPython.core.display.Javascript object>

In [9]:
dataset = create_numerframe(
    "test_assets/mini_numerai_version_1_data.csv", metadata={"version": 1}
)
copied_dataset = CopyPreProcessor().transform(dataset)
assert np.array_equal(copied_dataset.values, dataset.values)
assert dataset.meta == copied_dataset.meta

✅ Finished step CopyPreProcessor. Output shape=(10, 314). Time taken for step: 
0:00:00.003006. ✅

<IPython.core.display.Javascript object>

### 1.0.2. FeatureSelectionPreProcessor

`FeatureSelectionPreProcessor` will keep all features that you pass + keeps all other columns that are not features.

In [10]:
# export
@typechecked
class FeatureSelectionPreProcessor(BaseProcessor):
    """
    Keep only features given + all target, predictions and aux columns.
    """

    def __init__(self, feature_cols: Union[str, list]):
        super().__init__()
        self.feature_cols = feature_cols

    @display_processor_info
    def transform(self, dataf: NumerFrame) -> NumerFrame:
        keep_cols = (
            self.feature_cols
            + dataf.target_cols
            + dataf.prediction_cols
            + dataf.aux_cols
        )
        dataf = dataf.loc[:, keep_cols]
        return NumerFrame(dataf)

<IPython.core.display.Javascript object>

In [11]:
selected_dataset = FeatureSelectionPreProcessor(
    feature_cols=["feature_wisdom1"]
).transform(dataset)

assert selected_dataset.get_feature_data.shape[1] == 1
assert dataset.meta == selected_dataset.meta

✅ Finished step FeatureSelectionPreProcessor. Output shape=(10, 5). Time taken for step: 
0:00:00.001177. ✅

<IPython.core.display.Javascript object>

In [12]:
selected_dataset.head(2)

,feature_wisdom1,target,id,era,data_type
0,0.25,0.50,n000315175b67977,era1,train
1,0.50,0.25,n0014af834a96cdd,era1,train


<IPython.core.display.Javascript object>

### 1.0.3. TargetSelectionPreProcessor

`TargetSelectionPreProcessor` will keep all targets that you pass + all other columns that are not targets.

Not relevant for an inference pipeline, but especially convenient for Numerai Classic training if you train on a subset of the available targets. Can also be applied to Signals if you are using engineered targets in your pipeline.


In [13]:
# export
@typechecked
class TargetSelectionPreProcessor(BaseProcessor):
    """
    Keep only features given + all target, predictions and aux columns.
    """

    def __init__(self, target_cols: Union[str, list]):
        super().__init__()
        self.target_cols = target_cols

    @display_processor_info
    def transform(self, dataf: NumerFrame) -> NumerFrame:
        keep_cols = (
            self.target_cols
            + dataf.feature_cols
            + dataf.prediction_cols
            + dataf.aux_cols
        )
        dataf = dataf.loc[:, keep_cols]
        return NumerFrame(dataf)

<IPython.core.display.Javascript object>

In [14]:
dataset = create_numerframe(
    "test_assets/mini_numerai_version_2_data.parquet", metadata={"version": 2}
)
target_cols = ["target", "target_nomi_20", "target_nomi_60"]
selected_dataset = TargetSelectionPreProcessor(target_cols=target_cols).transform(
    dataset
)
assert selected_dataset.get_target_data.shape[1] == len(target_cols)
selected_dataset.head(2)

✅ Finished step TargetSelectionPreProcessor. Output shape=(10, 1055). Time taken for step: 
0:00:00.024541. ✅

,target,target_nomi_20,target_nomi_60,feature_dichasial_hammier_spawner,feature_rheumy_epistemic_prancer,feature_pert_performative_hormuz,feature_hillier_unpitied_theobromine,feature_perigean_bewitching_thruster,feature_renegade_undomestic_milord,feature_koranic_rude_corf,...,feature_drawable_exhortative_dispersant,feature_metabolic_minded_armorist,feature_investigatory_inerasable_circumvallation,feature_centroclinal_incentive_lancelet,feature_unemotional_quietistic_chirper,feature_behaviorist_microbiological_farina,feature_lofty_acceptable_challenge,feature_coactive_prefatorial_lucy,era,data_type
id,,,,,,,,,,,,,,,,,,,,,
n559bd06a8861222,0.25,0.25,0.50,0.25,0.75,0.25,0.75,0.25,0.50,1.0,...,1.00,0.0,0.0,0.25,0.00,0.0,1.00,0.25,0297,train
n9d39dea58c9e3cf,0.50,0.50,0.75,0.75,0.50,0.75,1.00,0.50,0.25,0.5,...,0.25,0.5,0.0,0.25,0.75,1.0,0.75,1.00,0003,train


<IPython.core.display.Javascript object>

### 1.0.4. ReduceMemoryProcessor

Numerai datasets can take up a lot of RAM and may put a strain on your compute environment.

For Numerai Classic, many of the feature and target columns can be downscaled to `float16`. `int8` if you are using the Numerai int8 datasets. For Signals it depends on the features you are generating.

`ReduceMemoryProcessor` downscales the type of your numeric columns to reduce the memory footprint as much as possible.

In [15]:
# export
class ReduceMemoryProcessor(BaseProcessor):
    """
    Reduce memory usage as much as possible.

    Credits to kainsama and others for writing about memory usage reduction for Numerai data:
    https://forum.numer.ai/t/reducing-memory/313

    :param deep_mem_inspect: Introspect the data deeply by interrogating object dtypes.
    Yields a more accurate representation of memory usage if you have complex object columns.
    """
    def __init__(self, deep_mem_inspect = False):
        super().__init__()
        self.deep_mem_inspect = deep_mem_inspect

    @display_processor_info
    def transform(self, dataf: Union[pd.DataFrame, NumerFrame]) -> NumerFrame:
        dataf = self._reduce_mem_usage(dataf)
        return NumerFrame(dataf)

    def _reduce_mem_usage(self, dataf: pd.DataFrame) -> pd.DataFrame:
        """
        Iterate through all columns and modify the numeric column types
        to reduce memory usage.
        """
        start_memory_usage = dataf.memory_usage(deep=self.deep_mem_inspect).sum() / 1024**2
        rich_print(f"Memory usage of DataFrame is [bold]{round(start_memory_usage, 2)} MB[/bold]")

        for col in dataf.columns:
            col_type = dataf[col].dtype.name

            if col_type not in ['object', 'category', 'datetime64[ns, UTC]','datetime64[ns]']:
                c_min = dataf[col].min()
                c_max = dataf[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        dataf[col] = dataf[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        dataf[col] = dataf[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        dataf[col] = dataf[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        dataf[col] = dataf[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        dataf[col] = dataf[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        dataf[col] = dataf[col].astype(np.float32)
                    else:
                        dataf[col] = dataf[col].astype(np.float64)

        end_memory_usage = dataf.memory_usage(deep=self.deep_mem_inspect).sum() / 1024**2
        rich_print(f"Memory usage after optimization is: [bold]{round(end_memory_usage, 2)} MB[/bold]")
        rich_print(f"[green] Usage decreased by [bold]{round(100 * (start_memory_usage - end_memory_usage) / start_memory_usage, 2)}%[/bold][/green]")
        return dataf

<IPython.core.display.Javascript object>

In [16]:
dataf = create_numerframe("test_assets/mini_numerai_version_2_data.parquet")
rmp = ReduceMemoryProcessor()
dataf = rmp.transform(dataf)

Memory usage of DataFrame is 0.04 MB

Memory usage after optimization is: 0.02 MB

 Usage decreased by 49.72%

✅ Finished step ReduceMemoryProcessor. Output shape=(10, 1073). Time taken for step: 
0:00:00.761351. ✅

<IPython.core.display.Javascript object>

In [17]:
#hide
dataf.head(2)

,era,data_type,feature_dichasial_hammier_spawner,feature_rheumy_epistemic_prancer,feature_pert_performative_hormuz,feature_hillier_unpitied_theobromine,feature_perigean_bewitching_thruster,feature_renegade_undomestic_milord,feature_koranic_rude_corf,feature_demisable_expiring_millepede,...,target_paul_20,target_paul_60,target_george_20,target_george_60,target_william_20,target_william_60,target_arthur_20,target_arthur_60,target_thomas_20,target_thomas_60
id,,,,,,,,,,,,,,,,,,,,,
n559bd06a8861222,0297,train,0.25,0.75,0.25,0.75,0.25,0.50,1.0,0.25,...,0.0,0.50,0.25,0.5,0.000000,0.500000,0.166626,0.500000,0.333252,0.500000
n9d39dea58c9e3cf,0003,train,0.75,0.50,0.75,1.00,0.50,0.25,0.5,0.00,...,0.5,0.75,0.50,0.5,0.666504,0.666504,0.500000,0.666504,0.500000,0.666504


<IPython.core.display.Javascript object>

## 1.1. Numerai Classic

The Numerai Classic dataset has a certain structure that you may not encounter in the Numerai Signals tournament.
Therefore, this section has all preprocessors that can only be applied to Numerai Classic.

### 1.1.0 Numerai Classic: Version agnostic

Preprocessors that work for all Numerai Classic versions.

In [18]:
# 1.1.0
# No version agnostic Numerai Classic preprocessors implemented yet.

<IPython.core.display.Javascript object>

### 1.1.1. Numerai Classic: Version 1 specific

Preprocessors that only work for version 1 (legacy data).
When using version 1 preprocessor it is recommended that the input `NumerFrame` has `version` in its metadata.
This avoids using version 1 preprocessors on version 2 data and encountering confusing error messages.

As a new user we recommend to start modeling the version 2 data and avoid version 1.
The preprocessors below are only there for legacy and compatibility reasons.

#### 1.1.1.1. GroupStatsPreProcessor

The version 1 legacy data has 6 groups of features which allows us to calculate aggregate features.

In [19]:
# export
class GroupStatsPreProcessor(BaseProcessor):
    """
    WARNING: Only supported for Version 1 (legacy) data. \n
    Calculate group statistics for all data groups. \n
    | :param groups: Groups to create features for. All groups by default.
    """

    def __init__(self, groups: list = None):
        super().__init__()
        self.all_groups = [
            "intelligence",
            "wisdom",
            "charisma",
            "dexterity",
            "strength",
            "constitution",
        ]
        self.group_names = groups if groups else self.all_groups

    @display_processor_info
    def transform(self, dataf: NumerFrame, *args, **kwargs) -> NumerFrame:
        """Check validity and add group features."""
        self._check_data_validity(dataf=dataf)
        dataf = dataf.pipe(self._add_group_features)
        return NumerFrame(dataf)

    def _add_group_features(self, dataf: pd.DataFrame) -> pd.DataFrame:
        """Mean, standard deviation and skew for each group."""
        for group in self.group_names:
            cols = [col for col in dataf.columns if group in col]
            dataf[f"feature_{group}_mean"] = dataf[cols].mean(axis=1)
            dataf[f"feature_{group}_std"] = dataf[cols].std(axis=1)
            dataf[f"feature_{group}_skew"] = dataf[cols].skew(axis=1)
        return dataf

    def _check_data_validity(self, dataf: NumerFrame):
        """Make sure this is only used for version 1 data."""
        assert hasattr(
            dataf.meta, "version"
        ), f"Version should be specified for '{self.__class__.__name__}' This Preprocessor will only work on version 1 data."
        assert (
            getattr(dataf.meta, "version") == 1
        ), f"'{self.__class__.__name__}' only works on version 1 data. Got version: '{getattr(dataf.meta, 'version')}'."

<IPython.core.display.Javascript object>

In [20]:
dataset = create_numerframe(
    "test_assets/mini_numerai_version_1_data.csv", metadata={"version": 1}
)
group_features_dataset = GroupStatsPreProcessor().transform(dataset)
group_features_dataset.head(2)
assert group_features_dataset.meta.version == 1

✅ Finished step GroupStatsPreProcessor. Output shape=(10, 332). Time taken for step: 
0:00:00.037703. ✅

<IPython.core.display.Javascript object>

In [21]:
# hide
new_cols = [
    "feature_intelligence_mean",
    "feature_intelligence_std",
    "feature_intelligence_skew",
    "feature_wisdom_mean",
    "feature_wisdom_std",
    "feature_wisdom_skew",
    "feature_charisma_mean",
    "feature_charisma_std",
    "feature_charisma_skew",
    "feature_dexterity_mean",
    "feature_dexterity_std",
    "feature_dexterity_skew",
    "feature_strength_mean",
    "feature_strength_std",
    "feature_strength_skew",
    "feature_constitution_mean",
    "feature_constitution_std",
    "feature_constitution_skew",
]
assert set(group_features_dataset.columns).intersection(new_cols)
group_features_dataset.get_feature_data[new_cols].head(2)

,feature_intelligence_mean,feature_intelligence_std,feature_intelligence_skew,feature_wisdom_mean,feature_wisdom_std,feature_wisdom_skew,feature_charisma_mean,feature_charisma_std,feature_charisma_skew,feature_dexterity_mean,feature_dexterity_std,feature_dexterity_skew,feature_strength_mean,feature_strength_std,feature_strength_skew,feature_constitution_mean,feature_constitution_std,feature_constitution_skew
0,0.333333,0.246183,0.558528,0.668478,0.236022,-0.115082,0.438953,0.259910,-0.004783,0.696429,0.200446,-0.607620,0.480263,0.292829,-0.372064,0.427632,0.27572,0.276155
1,0.208333,0.234359,0.382554,0.559783,0.358177,-0.062362,0.485465,0.252501,-0.021737,0.267857,0.249312,0.382267,0.407895,0.309866,0.220625,0.644737,0.33408,-0.794938


<IPython.core.display.Javascript object>

`GroupStatsPreProcessor` should break if `version != 1`.

In [22]:
# hide
def test_invalid_version(dataset: NumerFrame):
    copied_dataset = dataset.copy()
    copied_dataset.version = 2
    try:
        GroupStatsPreProcessor().transform(copied_dataset)
    except AssertionError:
        return True
    return False


test_invalid_version(dataset)

✅ Finished step GroupStatsPreProcessor. Output shape=(10, 332). Time taken for step: 
0:00:00.029719. ✅

False

<IPython.core.display.Javascript object>

### 1.1.2. Numerai Classic: Version 2 specific

Preprocessors that are only compatible with version 2 data. If the preprocessor is agnostic to Numerai Classic version implement under heading 1.1.0.

In [23]:
# 1.1.2
# No version 2 specific Numerai Classic preprocessors implemented yet.

<IPython.core.display.Javascript object>

## 1.2. Numerai Signals

Preprocessors that are specific to Numerai Signals.

### 1.2.1. TA-Lib Features (TalibFeatureGenerator)

[TA-Lib](https://mrjbq7.github.io/ta-lib) is an optimized technical analysis library. It is based on Cython and includes 150+ indicators. We have selected features based on feature importances, SHAP and correlation with the Numerai Signals target. If you want to implement other features check out the [TA-Lib documentation](https://mrjbq7.github.io/ta-lib/index.html).

Installation of TA-Lib is a bit more involved than just a pip install and is an optional dependency for this library. Visit the [installation documentation](https://mrjbq7.github.io/ta-lib/install.html) for instructions.

In [24]:
# export
class TalibFeatureGenerator(BaseProcessor):
    """
    Generate relevant features available in TA-Lib. \n
    More info: https://mrjbq7.github.io/ta-lib \n
    Input DataFrames for these functions should have the following columns defined:
    ['open', 'high', 'low', 'close', 'volume'] \n
    Make sure that all values are sorted in chronological order (by ticker). \n
    :param windows: List of ranges for window features.
    Windows will be applied for all features specified in self.window_features. \n
    :param ticker_col: Which column to groupby for feature generation.
    """

    def __init__(self, windows: List[int], ticker_col: str = "bloomberg_ticker"):
        self.__check_talib_import()
        super().__init__()

        self.windows = windows
        self.ticker_col = ticker_col
        self.window_features = [
            "NATR",
            "ADXR",
            "AROONOSC",
            "DX",
            "MFI",
            "MINUS_DI",
            "MINUS_DM",
            "MOM",
            "ROCP",
            "ROCR100",
            "PLUS_DI",
            "PLUS_DM",
            "BETA",
            "RSI",
            "ULTOSC",
            "TRIX",
            "ADXR",
            "CCI",
            "CMO",
            "WILLR",
        ]
        self.no_window_features = ["AD", "OBV", "APO", "MACD", "PPO"]
        self.hlocv_cols = ["open", "high", "low", "close", "volume"]

    def get_no_window_features(self, dataf: pd.DataFrame):
        for func in tqdm(self.no_window_features, desc="No window features"):
            dataf.loc[:, f"feature_{func}"] = (
                dataf.groupby(self.ticker_col)
                .apply(lambda x: pd.Series(self._no_window(x, func)).bfill())
                .values.astype(np.float32)
            )
        return dataf

    def get_window_features(self, dataf: pd.DataFrame):
        for win in tqdm(self.windows, position=0, desc="Window features"):
            for func in tqdm(self.window_features, position=1):
                dataf.loc[:, f"feature_{func}_{win}"] = (
                    dataf.groupby(self.ticker_col)
                    .apply(lambda x: pd.Series(self._window(x, func, win)).bfill())
                    .values.astype(np.float32)
                )
        return dataf

    def get_all_features(self, dataf: pd.DataFrame) -> pd.DataFrame:
        dataf = self.get_no_window_features(dataf)
        dataf = self.get_window_features(dataf)
        return dataf

    def transform(self, dataf: pd.DataFrame, *args, **kwargs) -> NumerFrame:
        return NumerFrame(self.get_all_features(dataf=dataf))

    def _no_window(self, dataf: pd.DataFrame, func) -> pd.Series:
        from talib import abstract as tab

        inputs = self.__get_inputs(dataf)
        if func in ["MACD"]:
            # MACD outputs tuple of 3 elements (value, signal and hist)
            return tab.Function(func)(inputs["close"])[0]
        else:
            return tab.Function(func)(inputs)

    def _window(self, dataf: pd.DataFrame, func, window: int) -> pd.Series:
        from talib import abstract as tab

        inputs = self.__get_inputs(dataf)
        if func in ["ULTOSC"]:
            # ULTOSC requires 3 timeperiods as input
            return tab.Function(func)(
                inputs["high"],
                inputs["low"],
                inputs["close"],
                timeperiod1=window,
                timeperiod2=window * 2,
                timeperiod3=window * 4,
            )
        else:
            return tab.Function(func)(inputs, timeperiod=window)

    def __get_inputs(self, dataf: pd.DataFrame) -> dict:
        return {col: dataf[col].values.astype(np.float64) for col in self.hlocv_cols}

    @staticmethod
    def __check_talib_import():
        try:
            from talib import abstract as tab
        except ImportError:
            raise ImportError(
                "TA-Lib is not installed for this environment. If you are using this class make sure to have TA-Lib installed. check https://mrjbq7.github.io/ta-lib/install.html for instructions on installation."
            )

<IPython.core.display.Javascript object>

In [25]:
# hide
# Example usage
# dataf = pd.DataFrame() # Your Signals DataFrame here.
# tfg = TalibFeatureGenerator(windows=[10, 20, 40], ticker_col="bloomberg_ticker")
# ta_dataf = tfg.transform(dataf=dataf)
# ta_dataf.head(2)

<IPython.core.display.Javascript object>

### 1.2.2. KatsuFeatureGenerator

[Katsu1110](https://www.kaggle.com/code1110) provides an excellent and fast feature engineering scheme in his [Kaggle notebook on starting with Numerai Signals](https://www.kaggle.com/code1110/numeraisignals-starter-for-beginners). It is surprisingly effective, fast and works well for modeling. This preprocessor is based on his feature engineering setup in that notebook.

Features generated:
1. MACD and MACD signal
2. RSI
3. Percentage rate of return
4. Volatility
5. MA (moving average) gap


In [26]:
# export
class KatsuFeatureGenerator(BaseProcessor):
    """
    Effective feature engineering setup based on Katsu's starter notebook.
    Based on source by Katsu1110: https://www.kaggle.com/code1110/numeraisignals-starter-for-beginners

    :param windows: Time interval to apply for window features: \n
    1. Percentage Rate of change \n
    2. Volatility \n
    3. Moving Average gap \n
    :param ticker_col: Columns with tickers to iterate over. \n
    :param close_col: Column name where you have closing price stored.
    """

    warnings.filterwarnings("ignore")

    def __init__(
        self,
        windows: list,
        ticker_col: str = "ticker",
        close_col: str = "close",
        num_cores: int = None,
    ):
        super().__init__()
        self.windows = windows
        self.ticker_col = ticker_col
        self.close_col = close_col
        self.num_cores = num_cores if num_cores else os.cpu_count()

    @display_processor_info
    def transform(self, dataf: Union[pd.DataFrame, NumerFrame]) -> NumerFrame:
        """Multiprocessing feature engineering."""
        tickers = dataf.loc[:, self.ticker_col].unique().tolist()
        rich_print(
            f"Feature engineering for {len(tickers)} tickers using {self.num_cores} CPU cores."
        )
        dataf_list = [
            x
            for _, x in tqdm(
                dataf.groupby(self.ticker_col), desc="Generating ticker DataFrames"
            )
        ]
        dataf = self._generate_features(dataf_list=dataf_list)
        return NumerFrame(dataf)

    def feature_engineering(self, dataf: pd.DataFrame) -> pd.DataFrame:
        """Feature engineering for single ticker."""
        close_series = dataf.loc[:, self.close_col]
        for x in self.windows:
            dataf.loc[
                :, f"feature_{self.close_col}_ROCP_{x}"
            ] = close_series.pct_change(x)

            dataf.loc[:, f"feature_{self.close_col}_VOL_{x}"] = (
                np.log1p(close_series).pct_change().rolling(x).std()
            )

            dataf.loc[:, f"feature_{self.close_col}_MA_gap_{x}"] = (
                close_series / close_series.rolling(x).mean()
            )

        dataf.loc[:, "feature_RSI"] = self._rsi(close_series)
        macd, macd_signal = self._macd(close_series)
        dataf.loc[:, "feature_MACD"] = macd
        dataf.loc[:, "feature_MACD_signal"] = macd_signal
        return dataf.bfill()

    def _generate_features(self, dataf_list: list) -> pd.DataFrame:
        """Add features for list of ticker DataFrames and concatenate."""
        with Pool(self.num_cores) as p:
            feature_datafs = list(
                tqdm(
                    p.imap(self.feature_engineering, dataf_list),
                    desc="Generating features",
                    total=len(dataf_list),
                )
            )
        return pd.concat(feature_datafs)

    @staticmethod
    def _rsi(close: pd.Series, period: int = 14) -> pd.Series:
        """
        See source https://github.com/peerchemist/finta
        and fix https://www.tradingview.com/wiki/Talk:Relative_Strength_Index_(RSI)
        """
        delta = close.diff()
        up, down = delta.copy(), delta.copy()
        up[up < 0] = 0
        down[down > 0] = 0

        gain = up.ewm(com=(period - 1), min_periods=period).mean()
        loss = down.abs().ewm(com=(period - 1), min_periods=period).mean()

        rs = gain / loss
        return pd.Series(100 - (100 / (1 + rs)))

    def _macd(
        self, close: pd.Series, span1=12, span2=26, span3=9
    ) -> Tuple[pd.Series, pd.Series]:
        """Compute MACD and MACD signal."""
        exp1 = self.__ema1(close, span1)
        exp2 = self.__ema1(close, span2)
        macd = 100 * (exp1 - exp2) / exp2
        signal = self.__ema1(macd, span3)
        return macd, signal

    @staticmethod
    def __ema1(series: pd.Series, span: int) -> pd.Series:
        """Exponential moving average"""
        a = 2 / (span + 1)
        return series.ewm(alpha=a).mean()

<IPython.core.display.Javascript object>

In [27]:
# other
from numerblox.download import KaggleDownloader
# Get price data from Kaggle
home_dir = "katsu_features_test"
kd = KaggleDownloader(home_dir)
kd.download_training_data("code1110/yfinance-stock-price-data-for-numerai-signals")

<IPython.core.display.Javascript object>

In [28]:
# other
dataf = create_numerframe(f"{home_dir}/full_data.parquet")
dataf.loc[:, 'friday_date'] = dataf['date']
# Take 500 ticker sample for test
dataf = dataf[dataf['ticker'].isin(dataf['ticker'].unique()[:500])]

<IPython.core.display.Javascript object>

In [29]:
# other
kfpp = KatsuFeatureGenerator(windows=[20, 40, 60], num_cores=8)
new_dataf = kfpp.transform(dataf)

Feature engineering for 500 tickers using 8 CPU cores.

Generating ticker DataFrames:   0%|          | 0/500 [00:00<?, ?it/s]

Generating features:   0%|          | 0/500 [00:00<?, ?it/s]

✅ Finished step KatsuFeatureGenerator. Output shape=(2519340, 21). Time taken for step: 
0:00:04.253744. ✅

<IPython.core.display.Javascript object>

12 features are generated in this test (3*3 window features + 3 non window features).

In [30]:
# other
new_dataf.sort_values(["ticker", "date"]).get_feature_data.tail(2)

,feature_close_ROCP_20,feature_close_VOL_20,feature_close_MA_gap_20,feature_close_ROCP_40,feature_close_VOL_40,feature_close_MA_gap_40,feature_close_ROCP_60,feature_close_VOL_60,feature_close_MA_gap_60,feature_RSI,feature_MACD,feature_MACD_signal
19644245,-0.086142,0.008137,0.981102,-0.158621,0.007118,0.923936,-0.16295,0.006266,0.890348,37.566483,-3.211289,-3.517807
19649114,-0.038835,0.007607,0.997180,-0.146552,0.007242,0.940975,-0.15529,0.006335,0.905626,42.684715,-2.901622,-3.394570


<IPython.core.display.Javascript object>

### 1.2.3. EraQuantileProcessor

In [31]:
# export
class EraQuantileProcessor(BaseProcessor):
    """
    Transform features into quantiles on a per-era basis

    :param num_quantiles: Number of buckets to split data into: \n
    :param era_col: Era column name in the dataframe to perform each transformation \n
    """

    def __init__(
        self,
        num_quantiles: int = 50,
        era_col: str = "friday_date",
        features: list = None,
        num_cores: int = None,
    ):
        super().__init__()
        self.num_quantiles = num_quantiles
        self.era_col = era_col
        self.num_cores = num_cores if num_cores else os.cpu_count()
        self.features = features

    def _process_eras(self, groupby_object):
        quantizer = QuantileTransformer(n_quantiles=self.num_quantiles, random_state=0)
        qt = lambda x: quantizer.fit_transform(x.values.reshape(-1, 1)).ravel()

        column = groupby_object.transform(qt)
        return column

    @display_processor_info
    def transform(
        self,
        dataf: Union[pd.DataFrame, NumerFrame],
    ) -> NumerFrame:
        """Multiprocessing quantile transforms by era."""
        self.features = self.features if self.features else dataf.feature_cols
        rich_print(
            f"Quantiling for {len(self.features)} features using {self.num_cores} CPU cores."
        )

        date_groups = dataf.groupby(self.era_col)
        groupby_objects = [date_groups[feature] for feature in self.features]

        with Pool() as p:
            results = list(
                tqdm(
                    p.imap(self._process_eras, groupby_objects),
                    total=len(groupby_objects),
                )
            )

        quantiles = pd.concat(results, axis=1)
        dataf[[f"{feature}_quantile" for feature in self.features]] = quantiles
        return NumerFrame(dataf)

<IPython.core.display.Javascript object>

In [32]:
# other
era_quantiler = EraQuantileProcessor(num_quantiles=50)
era_dataf = era_quantiler.transform(new_dataf)

Quantiling for 12 features using 16 CPU cores.

  0%|          | 0/12 [00:00<?, ?it/s]

✅ Finished step EraQuantileProcessor. Output shape=(2519340, 33). Time taken for step: 
0:01:55.945342. ✅

<IPython.core.display.Javascript object>

In [33]:
# other
era_dataf.get_feature_data.tail(2)

,feature_close_ROCP_20,feature_close_VOL_20,feature_close_MA_gap_20,feature_close_ROCP_40,feature_close_VOL_40,feature_close_MA_gap_40,feature_close_ROCP_60,feature_close_VOL_60,feature_close_MA_gap_60,feature_RSI,...,feature_close_MA_gap_20_quantile,feature_close_ROCP_40_quantile,feature_close_VOL_40_quantile,feature_close_MA_gap_40_quantile,feature_close_ROCP_60_quantile,feature_close_VOL_60_quantile,feature_close_MA_gap_60_quantile,feature_RSI_quantile,feature_MACD_quantile,feature_MACD_signal_quantile
19644245,-0.086142,0.008137,0.981102,-0.158621,0.007118,0.923936,-0.16295,0.006266,0.890348,37.566483,...,0.186816,0.070566,0.856538,0.066978,0.101842,0.844413,0.070784,0.060159,0.09877,0.098609
19649114,-0.038835,0.007607,0.997180,-0.146552,0.007242,0.940975,-0.15529,0.006335,0.905626,42.684715,...,0.223813,0.069825,0.941025,0.097017,0.121483,0.938354,0.087984,0.093973,0.10318,0.119625


<IPython.core.display.Javascript object>

In [34]:
# other
# hide
kd.remove_base_directory()

⚠ Deleting directory for 'KaggleDownloader' ⚠
Path: 
'/Users/clepelaars/Desktop/crowdcent/repositories/numerai-blocks/nbs/katsu_features_test'

<IPython.core.display.Javascript object>

## 2. Custom preprocessors

There are an almost unlimited number of ways to preprocess (selection, engineering and manipulation). We have only scratched the surface with the preprocessors currently implemented. We invite the Numerai community to develop Numerai Classic and Numerai Signals preprocessors.

A new Preprocessor should inherit from `BaseProcessor` and implement a `transform` method. For efficient implementation, we recommend you use `NumerFrame` functionality for preprocessing. You can also support Pandas DataFrame input as long as the `transform` method returns a `NumerFrame`. This ensures that the Preprocessor still works within a full `numerai-blocks` pipeline. A template for new preprocessors is given below.

To enable fancy logging output. Add the `@display_processor_info` decorator to the `transform` method.

In [35]:
# export
class AwesomePreProcessor(BaseProcessor):
    """ TEMPLATE - Do some awesome preprocessing. """
    def __init__(self):
        super().__init__()

    @display_processor_info
    def transform(self, dataf: NumerFrame, *args, **kwargs) -> NumerFrame:
        # Do processing
        ...
        # Parse all contents of NumerFrame to the next pipeline step
        return NumerFrame(dataf)

<IPython.core.display.Javascript object>

-------------------------------------------

In [36]:
# hide
# Run this cell to sync all changes with library
from nbdev.export import notebook2script

notebook2script()

Converted 00_misc.ipynb.
Converted 01_download.ipynb.
Converted 02_numerframe.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04_model.ipynb.
Converted 05_postprocessing.ipynb.
Converted 06_modelpipeline.ipynb.
Converted 07_evaluation.ipynb.
Converted 08_key.ipynb.
Converted 09_submission.ipynb.
Converted 10_staking.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>